# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

DB = "prices.db"

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [4]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [5]:
get_ticket_price("Paris")

DATABASE TOOL CALLED: Getting price for Paris


'Ticket price to Paris is $899.0'

In [6]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": price_function}]
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [ ]:

def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

In [7]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content

In [ ]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

## A bit more about what Gradio actually does:

1. Gradio constructs a frontend Svelte app based on our Python description of the UI
2. Gradio starts a server built upon the Starlette web framework listening on a free port that serves this React app
3. Gradio creates backend routes for our callbacks, like chat(), which calls our functions

And of course when Gradio generates the frontend app, it ensures that the the Submit button calls the right backend route.

That's it!

It's simple, and it has a result that feels magical.

# Let's go multi-modal!!

We can use DALL-E-3, the image generation model behind GPT-4o, to make us some images

Let's put this in a function called artist.

### Price alert: each time I generate an image it costs about 4 cents - don't go crazy with images!

In [ ]:
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image

In [ ]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
image = artist("New York City")
display(image)

In [ ]:
def talker(message):
    response = openai.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="onyx",    # Also, try replacing onyx with alloy or coral
      input=message
    )
    return response.content

## Let's bring this home:

1. A multi-modal AI assistant with image and audio generation
2. Tool callling with database lookup
3. A step towards an Agentic workflow


In [5]:
def chat(history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    cities = []
    image = None

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses, cities = handle_tool_calls_and_return_cities(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    voice = talker(reply)

    if cities:
        image = artist(cities[0])
    
    return history, voice, image


In [4]:
def handle_tool_calls_and_return_cities(message):
    responses = []
    cities = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            cities.append(city)
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses, cities

## The 3 types of Gradio UI

`gr.Interface` is for standard, simple UIs

`gr.ChatInterface` is for standard ChatBot UIs

`gr.Blocks` is for custom UIs where you control the components and the callbacks

In [ ]:
# Callbacks (along with the chat() function above)

def put_message_in_chatbot(message, history):
        return "", history + [{"role":"user", "content":message}]

# UI definition

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")

# Hooking up events to callbacks

    message.submit(put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, audio_output, image_output]
    )

ui.launch(inbrowser=True, auth=("ed", "bananas"))

# Exercises and Business Applications

Add in more tools - perhaps to simulate actually booking a flight. A student has done this and provided their example in the community contributions folder.

Next: take this and apply it to your business. Make a multi-modal AI assistant with tools that could carry out an activity for your work. A customer support assistant? New employee onboarding assistant? So many possibilities! Also, see the week2 end of week Exercise in the separate Notebook.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a HUGE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

In [ ]:
# Complete Multimodal AI Assistant with Booking Capability

import json
import tempfile
import time
from datetime import datetime

# Helper function to save audio to file for Gradio
def talker_to_file(message):
    """Generate speech from text and return the path to the audio file for Gradio"""
    try:
        response = openai.audio.speech.create(
            model="gpt-4o-mini-tts",
            voice="onyx",
            input=message
        )
        
        # Save to a temporary file
        timestamp = int(time.time() * 1000)
        output_filename = f"audio_output_{timestamp}.mp3"
        
        with open(output_filename, "wb") as f:
            f.write(response.content)
        
        return output_filename
    except Exception as e:
        print(f"Error generating speech: {e}")
        return None

# Create bookings table if it doesn't exist
def init_bookings_table():
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS bookings (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                city TEXT NOT NULL,
                passenger_name TEXT NOT NULL,
                travel_date TEXT,
                booking_date TEXT NOT NULL,
                status TEXT DEFAULT 'confirmed',
                price REAL
            )
        ''')
        conn.commit()

init_bookings_table()

# Enhanced system message that encourages booking
system_message_booking = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1-2 sentences.
Always be accurate. If you don't know the answer, say so.
When a customer asks about prices, offer to help them book a flight.
If they want to book, make sure to ask for their name and preferred travel date.
Always confirm bookings enthusiastically with "Booking confirmed" in your reply.
"""

# Booking function
def book_ticket(destination_city, passenger_name, travel_date):
    """Book a ticket for a passenger to a destination city"""
    print(f"DATABASE TOOL CALLED: Booking ticket for {passenger_name} to {destination_city} on {travel_date}", flush=True)
    
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        # First check if city exists and get price
        cursor.execute('SELECT price FROM prices WHERE city = ?', (destination_city.lower(),))
        price_result = cursor.fetchone()
        
        if not price_result:
            return f"Sorry, we don't have flights to {destination_city.title()} at this time."
        
        price = price_result[0]
        booking_date = datetime.now().strftime('%Y-%m-%d')
        
        # Insert booking
        cursor.execute('''
            INSERT INTO bookings (city, passenger_name, travel_date, booking_date, status, price)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (destination_city.lower(), passenger_name, travel_date, booking_date, 'confirmed', price))
        
        booking_id = cursor.lastrowid
        conn.commit()
        
        return f"Booking confirmed! Booking ID: {booking_id}. Passenger: {passenger_name}, Destination: {destination_city.title()}, Travel Date: {travel_date}, Price: ${price:.2f}"

# Booking tool definition
booking_function = {
    "name": "book_ticket",
    "description": "Book a flight ticket for a passenger to a destination city. Use this when the customer wants to make a booking.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The destination city the customer wants to travel to",
            },
            "passenger_name": {
                "type": "string",
                "description": "The full name of the passenger making the booking",
            },
            "travel_date": {
                "type": "string",
                "description": "The desired travel date in YYYY-MM-DD format",
            },
        },
        "required": ["destination_city", "passenger_name", "travel_date"],
        "additionalProperties": False
    }
}

# Combine all tools (use the existing price_function and add booking)
all_tools_booking = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": booking_function}
]

# Enhanced handle tool calls function
def handle_tool_calls_with_booking(message):
    responses = []
    cities = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            cities.append(city)
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "book_ticket":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            passenger_name = arguments.get('passenger_name')
            travel_date = arguments.get('travel_date')
            cities.append(city)
            booking_result = book_ticket(city, passenger_name, travel_date)
            responses.append({
                "role": "tool",
                "content": booking_result,
                "tool_call_id": tool_call.id
            })
    return responses, cities

# Enhanced chat function with booking support bug checked
def chat_with_booking(history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message_booking}] + history
    response = openai.chat.completions.create(
        model=MODEL, 
        messages=messages, 
        tools=all_tools_booking
    )
    cities = []
    image = None
    audio_file = None

    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses, cities = handle_tool_calls_with_booking(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(
            model=MODEL, 
            messages=messages, 
            tools=all_tools_booking
        )

    reply = response.choices[0].message.content
    history += [{"role": "assistant", "content": reply}]

    # Generate audio for the response (save to file for Gradio)
    audio_file = talker_to_file(reply)

    # Generate image for the first city mentioned or booked
    if cities:
        image = artist(cities[0])

    return history, audio_file, image

# Callback function to handle message input
def put_message_in_chatbot_booking(message, history):
    return "", history + [{"role": "user", "content": message}]

# Complete Multimodal Gradio Interface
with gr.Blocks(title="FlightAI - Multimodal Booking Assistant") as multimodal_ui:
    gr.Markdown("# 🛫 FlightAI - Multimodal AI Booking Assistant")
    gr.Markdown("Ask about flight prices and book tickets! I can show you images and speak to you.")
    
    with gr.Row():
        chatbot = gr.Chatbot(
            height=500, 
            type="messages",
            label="Chat"
        )
        image_output = gr.Image(
            height=500, 
            interactive=False,
            label="Destination Image"
        )
    
    with gr.Row():
        audio_output = gr.Audio(
            autoplay=True,
            label="Audio Response"
        )
    
    with gr.Row():
        message = gr.Textbox(
            label="Type your message:",
            placeholder="Ask about flight prices or book a ticket..."
        )

    # Connect events: submit message -> update chat -> process -> return results
    message.submit(
        put_message_in_chatbot_booking, 
        inputs=[message, chatbot], 
        outputs=[message, chatbot]
    ).then(
        chat_with_booking, 
        inputs=chatbot, 
        outputs=[chatbot, audio_output, image_output]
    )

# Launch the interface
multimodal_ui.launch(inbrowser=True)
# Complete Multimodal AI Assistant with Booking Capability

from datetime import datetime

# Create bookings table if it doesn't exist
def init_bookings_table():
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS bookings (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                city TEXT NOT NULL,
                passenger_name TEXT NOT NULL,
                travel_date TEXT,
                booking_date TEXT NOT NULL,
                status TEXT DEFAULT 'confirmed',
                price REAL
            )
        ''')
        conn.commit()

init_bookings_table()

# Enhanced system message that encourages booking
system_message_booking = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1-2 sentences.
Always be accurate. If you don't know the answer, say so.
When a customer asks about prices, offer to help them book a flight.
If they want to book, make sure to ask for their name and preferred travel date.
Always confirm bookings enthusiastically with "Booking confirmed" in your reply.
"""

# Booking function
def book_ticket(destination_city, passenger_name, travel_date):
    """Book a ticket for a passenger to a destination city"""
    print(f"DATABASE TOOL CALLED: Booking ticket for {passenger_name} to {destination_city} on {travel_date}", flush=True)
    
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        # First check if city exists and get price
        cursor.execute('SELECT price FROM prices WHERE city = ?', (destination_city.lower(),))
        price_result = cursor.fetchone()
        
        if not price_result:
            return f"Sorry, we don't have flights to {destination_city.title()} at this time."
        
        price = price_result[0]
        booking_date = datetime.now().strftime('%Y-%m-%d')
        
        # Insert booking
        cursor.execute('''
            INSERT INTO bookings (city, passenger_name, travel_date, booking_date, status, price)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (destination_city.lower(), passenger_name, travel_date, booking_date, 'confirmed', price))
        
        booking_id = cursor.lastrowid
        conn.commit()
        
        return f"Booking confirmed! Booking ID: {booking_id}. Passenger: {passenger_name}, Destination: {destination_city.title()}, Travel Date: {travel_date}, Price: ${price:.2f}"

# Updated tool definitions
booking_function = {
    "name": "book_ticket",
    "description": "Book a flight ticket for a passenger to a destination city. Use this when the customer wants to make a booking.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The destination city the customer wants to travel to",
            },
            "passenger_name": {
                "type": "string",
                "description": "The full name of the passenger making the booking",
            },
            "travel_date": {
                "type": "string",
                "description": "The desired travel date in YYYY-MM-DD format",
            },
        },
        "required": ["destination_city", "passenger_name", "travel_date"],
        "additionalProperties": False
    }
}

# Combine all tools to work togerther
all_tools_booking = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": booking_function}
]

# Enhanced handle tool calls function
def handle_tool_calls_with_booking(message):
    responses = []
    cities = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            cities.append(city)
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "book_ticket":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            passenger_name = arguments.get('passenger_name')
            travel_date = arguments.get('travel_date')
            cities.append(city)
            booking_result = book_ticket(city, passenger_name, travel_date)
            responses.append({
                "role": "tool",
                "content": booking_result,
                "tool_call_id": tool_call.id
            })
    return responses, cities

# Enhanced chat function with booking support
def chat_with_booking(history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message_booking}] + history
    response = openai.chat.completions.create(
        model=MODEL, 
        messages=messages, 
        tools=all_tools_booking
    )
    cities = []
    image = None
    audio_file = None

    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses, cities = handle_tool_calls_with_booking(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(
            model=MODEL, 
            messages=messages, 
            tools=all_tools_booking
        )

    reply = response.choices[0].message.content
    history += [{"role": "assistant", "content": reply}]

    # Generate audio for the response (save to file for Gradio)
    audio_file = talker_to_file(reply)

    # Generate image for the first city mentioned or booked
    if cities:
        image = artist(cities[0])

    return history, audio_file, image

# Callback function to handle message input
def put_message_in_chatbot(message, history):
    return "", history + [{"role": "user", "content": message}]

# Complete Multimodal Gradio Interface
with gr.Blocks(title="FlightAI - Multimodal Booking Assistant") as multimodal_ui:
    gr.Markdown("# 🛫 FlightAI - Multimodal AI Booking Assistant")
    gr.Markdown("Ask about flight prices and book tickets! I can show you images and speak to you.")
    
    with gr.Row():
        chatbot = gr.Chatbot(
            height=500, 
            type="messages",
            label="Chat"
        )
        image_output = gr.Image(
            height=500, 
            interactive=False,
            label="Destination Image"
        )
    
    with gr.Row():
        audio_output = gr.Audio(
            autoplay=True,
            label="Audio Response"
        )
    
    with gr.Row():
        message = gr.Textbox(
            label="Type your message:",
            placeholder="Ask about flight prices or book a ticket..."
        )

    # Connect events: submit message -> update chat -> process -> return results
    message.submit(
        put_message_in_chatbot, 
        inputs=[message, chatbot], 
        outputs=[message, chatbot]
    ).then(
        chat_with_booking, 
        inputs=chatbot, 
        outputs=[chatbot, audio_output, image_output]
    )

# Launch the interface
multimodal_ui.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
